In [1]:
from generator import load_llama3_tokenizer
from datasets import load_dataset
from train import *
import json
import os
from tqdm import tqdm
from multiprocess import Pool
import itertools

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[2025-05-19 12:05:05,109] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
mimi = None
device = 'cpu'
text_tokenizer = load_llama3_tokenizer()

dataset = load_dataset('mesolitica/Malaysian-Emilia-Sesame')['malaysian_podcast']

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [3]:
%%time

data = dataset[-1]
segment_tokens, segment_tokens_mask = get_input_ids(
    data['reference_audio'], 
    data['reference_text'],
    data['target_audio'], 
    data['target_text'],
    mimi=mimi,
    text_tokenizer=text_tokenizer,
    device=device,
    calculated_speech_tokens=True,
)
segment_tokens.shape[0]

CPU times: user 50.1 ms, sys: 3.08 ms, total: 53.2 ms
Wall time: 10.6 ms


407

In [4]:
def loop(indices):
    indices, _ = indices
    lengths = []
    
    dataset = load_dataset('mesolitica/Malaysian-Emilia-Sesame')['malaysian_podcast']
    text_tokenizer = load_llama3_tokenizer()
    for i in tqdm(indices):
        data = dataset[i]
        segment_tokens, segment_tokens_mask = get_input_ids(
            data['reference_audio'], 
            data['reference_text'],
            data['target_audio'], 
            data['target_text'],
            mimi=mimi,
            text_tokenizer=text_tokenizer,
            device=device,
            calculated_speech_tokens=True,
        )
        lengths.append({
            'i': i,
            'length': segment_tokens.shape[0]
        })
    return lengths

In [5]:
lengths = multiprocessing(range(len(dataset)), loop, cores = 20)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 413.65it/s]


In [6]:
rows = sorted(lengths, key = lambda x: x['length'])

In [17]:
maxlen = 2048
data, temp, l = [], [], 0
for r in tqdm(rows):
    if l + r['length'] >= maxlen:
        data.append(temp)
        temp = [r['i']]
        l = r['length']
    else:
        l += r['length']
        temp.append(r['i'])
if len(temp):
    data.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 359236/359236 [00:00<00:00, 1233834.40it/s]


In [23]:
import json

with open('merged.json', 'w') as fopen:
    json.dump(data, fopen)

In [24]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="merged.json",
    path_in_repo="merged-podcast.json",
    repo_id="mesolitica/Malaysian-Emilia-Audio-Tokens",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Emilia-Audio-Tokens/commit/dd843c0f2c349c4330dc14cd67507014f2db7578', commit_message='Upload merged-podcast.json with huggingface_hub', commit_description='', oid='dd843c0f2c349c4330dc14cd67507014f2db7578', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Emilia-Audio-Tokens', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Emilia-Audio-Tokens'), pr_revision=None, pr_num=None)